Автор:     Шестаков Михаил Сергеевич  
Telegram:  https://t.me/mshestakov1  
Телефон:   +7(950)035-55-38  
email:     mikhail-shestakov-2022@bk.ru  

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT LOWER(us.user_id) AS user_id,
       us.lt_day,
       CASE
           WHEN us.lt_day <= 365 THEN "New user"
           Else 'Old user'
       END AS is_new,
       CAST(us.age AS INT) AS age,
       CASE
           WHEN us.gender_segment LIKE '0%' THEN 'Man'
           WHEN us.gender_segment LIKE '1%' THEN 'Woman'
       END AS gender_segment,
       LOWER(us.os_name) AS os_name,
       LOWER(us.cpe_type_name) AS cpe_type_name,
       l.country,
       l.city,
       aseg.age_segment,
       tseg.traffic_segment,
       lseg.lifetime_segment,
       us.nps_score,
       CASE
           WHEN us.nps_score >= 0 AND us.nps_score <= 6 THEN 'detractors'
           WHEN us.nps_score >= 7 AND us.nps_score <= 8 THEN 'passives'
           WHEN us.nps_score >= 9 AND us.nps_score <= 10 THEN 'promoters'
       END AS nps_group

FROM user AS us
INNER JOIN location AS l ON us.location_id=l.location_id
INNER JOIN (SELECT age_gr_id,
                   title, substr(title, 4) AS age_segment
            FROM age_segment
            WHERE title NOT LIKE '08%') AS aseg ON aseg.age_gr_id=us.age_gr_id
INNER JOIN (SELECT tr_gr_id,
                   title, substr(title, 4) AS traffic_segment
            FROM traffic_segment) AS tseg ON tseg.tr_gr_id=us.tr_gr_id
INNER JOIN (SELECT lt_gr_id,
                   title, substr(title, 4) AS lifetime_segment
            FROM lifetime_segment) AS lseg ON lseg.lt_gr_id=us.lt_gr_id
WHERE us.gender_segment LIKE '0%'
      OR us.gender_segment LIKE '1%';           
"""

In [4]:
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,a001a2,2320,Old user,45,Woman,android,smartphone,Россия,Уфа,45-54,1-5,36+,10,promoters
1,a001wf,2344,Old user,53,Man,android,smartphone,Россия,Киров,45-54,1-5,36+,10,promoters
2,a003q7,467,Old user,57,Man,android,smartphone,Россия,Москва,55-64,20-25,13-24,10,promoters
3,a004tb,4190,Old user,44,Woman,ios,smartphone,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,a004xt,1163,Old user,24,Man,android,smartphone,Россия,Рязань,16-24,5-10,36+,10,promoters


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Ссылка на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/.13981647/viz/telecomm_csi_tableau_16470850661180/Dashboard1?publish=yes

### Ссылка на pdf-файл с презентацией:

https://disk.yandex.ru/i/MYLekah6tNnX3g